In [1]:
#
# Import Libraries
#

import pandas as pd
from dotmap import DotMap
import os

In [3]:
#
# Constants
#

modes = ['C18N', 'C18P']

# mfilt_path = r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Analysis\00-FeatureSelection\ALDH4\mfilt.tsv"

f2i_path = r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\ALDH4\WorkingFiles\f2i.tsv"

alid_path = r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\ALDH4\OriginalFiles\RBR_identifications.xlsx"

tp_path = r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\ALDH4\WorkingFiles\Identifications\TP_results"

In [5]:
f2i = pd.read_csv(f2i_path, sep='\t')

In [38]:
tpm = DotMap()
tpm.cn = pd.read_csv(os.path.join(tp_path, 'C18N/4_TPFilter_CMM_cn.tsv'), sep='\t')
tpm.cp = pd.read_csv(os.path.join(tp_path, 'C18P/4_TPFilter_CMM_cp.tsv'), sep='\t')

In [39]:
tpm = pd.concat([tpm.cp, tpm.cn]).loc[:, [i for i in tpm.cn.columns if 'seqn_' not in i]]

In [40]:
TP_columns = [
    'FeatureInfo_Name',
    'Name',
    'Peptide',
    'Halogenated',
    'Plant',
    'NaturalProduct',
    'MDM',
    'Drug',
    'Food',
    'Adduct',
    'TP_Class_argmax',
    'TPMetrics'
]

tpm = tpm.loc[:, TP_columns].drop_duplicates()

In [42]:
tpm = tpm.fillna('')\
        .groupby('FeatureInfo_Name')\
            .agg(lambda x: ' | '.join([str(i) for i in list(x)]))\
                .reset_index()\
                    .rename(columns={'Name': 'TP_ID', 'Adduct':'TP_Adduct', 'FeatureInfo_Name':'fid'})

In [48]:
f2i_tp = pd.merge(
    f2i,
    tpm,
    how='outer',
    on='fid'
)

In [55]:
f2i_tp.to_csv('f2i_TP.tsv', sep='\t', index=False)